In [1]:
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import save_model, load_model
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Load data
with h5py.File('/content/drive/MyDrive/Colab Notebooks/data.h5', 'r') as f:
    velocity_data = f['velocity'][:]
    density_data = f['density'][:]
    boundary_data = f['boundary'][:]

velocity_data = np.expand_dims(velocity_data, axis=-1)  # Shape: (4002, 100, 400, 1)
density_data = np.expand_dims(density_data, axis=-1)    # Shape: (4002, 100, 400, 1)
boundary_data = np.expand_dims(boundary_data, axis=-1)  # Shape: (4002, 100, 400, 1)

print("Velocity Data Shape:", velocity_data.shape)
print("Density Data Shape:", density_data.shape)
print("Boundary Data Shape:", boundary_data.shape)

Velocity Data Shape: (8004, 100, 400, 1)
Density Data Shape: (8004, 100, 400, 1)
Boundary Data Shape: (8004, 100, 400, 1)


In [3]:
# Reshape the data to 2D for scaling
num_samples = velocity_data.shape[0]
H, W = velocity_data.shape[1], velocity_data.shape[2]
velocity_data_2d = velocity_data.reshape(num_samples, -1)
density_data_2d = density_data.reshape(num_samples, -1)
boundary_data_2d = boundary_data.reshape(num_samples, -1)

# Create and fit scalers
velocity_scaler = StandardScaler().fit(velocity_data_2d)
density_scaler = StandardScaler().fit(density_data_2d)
boundary_scaler = StandardScaler().fit(boundary_data_2d)

# Transform the data
velocity_data_scaled = velocity_scaler.transform(velocity_data_2d).reshape(num_samples, H, W)
density_data_scaled = density_scaler.transform(density_data_2d).reshape(num_samples, H, W)
boundary_data_scaled = boundary_scaler.transform(boundary_data_2d).reshape(num_samples, H, W)

# Similarly, you should also fit and transform the target data


In [9]:
# Combine inputs: velocity, density, and boundary
X = np.concatenate((velocity_data_scaled, density_data_scaled, boundary_data_scaled), axis=-1)  # Shape: (4002, 100, 400, 3)
# Targets: velocity and density
y = np.concatenate((velocity_data_scaled, density_data_scaled), axis=-1)  # Shape: (4002, 100, 400, 2)

X = np.concatenate((velocity_data_scaled[:-1], density_data_scaled[:-1], boundary_data_scaled[:-1]), axis=-1)  # Exclude last time step
y = np.concatenate((velocity_data_scaled[1:], density_data_scaled[1:]), axis=-1)  # Exclude first time step


In [8]:
print(X.shape)

(8003, 100, 400, 3)


In [5]:
# Combine inputs: velocity, density, and boundary
X = np.concatenate((velocity_data, density_data, boundary_data), axis=-1)  # Shape: (4002, 100, 400, 3)
# Targets: velocity and density
y = np.concatenate((velocity_data, density_data), axis=-1)  # Shape: (4002, 100, 400, 2)

X = np.concatenate((velocity_data[:-1], density_data[:-1], boundary_data[:-1]), axis=-1)  # Exclude last time step
y = np.concatenate((velocity_data[1:], density_data[1:]), axis=-1)  # Exclude first time step


In [ ]:
# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [7]:
print(X_train.shape)

(5602, 100, 400, 3)


In [7]:
# Define the CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(y_train.shape[1] * y_train.shape[2] * y_train.shape[3], activation='linear'),  # Output layer
    layers.Reshape((y_train.shape[1], y_train.shape[2], y_train.shape[3]))  # Adjust based on your target shape
])



IndexError: tuple index out of range

In [ ]:
# Compile the model
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])


# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])



In [ ]:
# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

model.save('velocity_density_predictor.keras')

In [ ]:
model.summary()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

print('Model size: %dKB' % (len(tflite_model) / 1024))

with open('velocity_density_predictorCyl.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
modelT = load_model('velocity_density_predictor.keras')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation, FFMpegWriter
from matplotlib.widgets import Button

NEXT Thing

In [ ]:
def distance(x1,y1,x2,y2):
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def toggle_visibility(event):
    if vorticity_img.get_visible():
        vorticity_img.set_visible(False)
        velocity_img.set_visible(True)
    else:
        vorticity_img.set_visible(True)
        velocity_img.set_visible(False)
    fig.canvas.draw_idle()

# Simulation parameters
Nx = 400    # resolution x-dir
Ny = 100    # resolution y-dir
rho0 = 100  # average density
tau = 0.6   # collision timescale
Nt = 1000   # number of timesteps
plotRealTime = True  # switch on for plotting as the simulation goes along

# Lattice speeds / weights
NL = 9
idxs = np.arange(NL)
cxs = np.array([0, 0, 1, 1, 1, 0, -1, -1, -1])
cys = np.array([0, 1, 1, 0, -1, -1, -1, 0, 1])
weights = np.array([4/9, 1/9, 1/36, 1/9, 1/36, 1/9, 1/36, 1/9, 1/36])

# Initial Conditions
F = np.ones((Ny, Nx, NL))  # * rho0 / NL
F += 0.01 * np.random.randn(Ny, Nx, NL)  # Perturb initial state
F[:, :, 3] += 2.3 #2 * (1 + 0.2 * np.cos(2 * np.pi * X / Nx * 4))

rho = np.sum(F, 2)
for i in idxs:
    F[:, :, i] *= rho0 / rho

# Cylinder boundary
circleX, circleY = Nx//4, Ny//2
radius = 13
objects = np.full((Ny,Nx),False)
for y in range(Ny):
    for x in range(Nx):
        if distance(circleX,circleY,x,y) < radius: objects[y][x] = True


# Prep figure
fig, ax = plt.subplots(figsize=(8, 4), dpi=80)
ax.set_aspect('equal')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

# Initialize the plot with empty data
vorticity_img = ax.imshow(np.zeros((Ny, Nx)), cmap='bwr', animated=True)
velocity_img = ax.imshow(np.zeros((Ny, Nx)), cmap='viridis', animated=True)
cylinder_img = ax.imshow(~objects, cmap='gray', alpha=0.3, animated=True)

velocity_img.set_visible(False)
vorticity_img.set_visible(True)

In [ ]:
def update(it):
    global F

    # Prepare input for the model
    rho = np.sum(F, 2)
    ux = np.sum(F * cxs, 2) / rho
    uy = np.sum(F * cys, 2) / rho
    velocity = np.sqrt(ux**2 + uy**2)

    # Reshape input data for model
    model_input = np.stack([velocity, rho, objects.astype(np.float32)], axis=-1)
    model_input = np.expand_dims(model_input, axis=0)  # Add batch dimension
    # Predict the next state
    prediction = modelT.predict(model_input)

    # Extract predicted velocity and density
    predicted_velocity = prediction[0, :, :, 0]  # Adjust according to your model's output shape
    predicted_density = prediction[0, :, :, 1]  # Adjust according to your model's output shape

    # Update the current state with the predicted state
    velocity_img.set_array(predicted_velocity)
    velocity_img.set_clim(0, np.max(predicted_velocity))

    vorticity = (np.roll(ux, -1, axis=0) - np.roll(ux, 1, axis=0)) - (np.roll(uy, -1, axis=1) - np.roll(uy, 1, axis=1))
    vorticity[objects] = np.nan
    vorticity = np.ma.array(vorticity, mask=objects)

    vorticity_img.set_array(vorticity)
    vorticity_img.set_clim(-0.1, 0.1)

    return velocity_img, vorticity_img, cylinder_img

In [ ]:
def updateOG(it):
    """ Update function for animation """
    global F

    F[:, -1, [6,7,8]] = F[:, -2, [6,7,8]]
    F[:, 0, [2,3,4]] = F[:, 1, [2,3,4]]
    # Drift
    for i, cx, cy in zip(idxs, cxs, cys):
        F[:, :, i] = np.roll(F[:, :, i], cx, axis=1)  # Shift by cx in x-direction
        F[:, :, i] = np.roll(F[:, :, i], cy, axis=0)  # Shift by cy in y-direction

    # Set reflective boundaries
    bndryF = F[objects, :]
    bndryF = bndryF[:, [0, 5, 6, 7, 8, 1, 2, 3, 4]]

    # Calculate fluid variables
    rho = np.sum(F, 2)
    ux = np.sum(F * cxs, 2) / rho
    uy = np.sum(F * cys, 2) / rho

    # Apply Collision
    Feq = np.zeros(F.shape)
    for i, cx, cy, w in zip(idxs, cxs, cys, weights):
        Feq[:, :, i] = rho * w * (1 + 3 * (cx * ux + cy * uy) + 9 * (cx * ux + cy * uy)**2 / 2 - 3 * (ux**2 + uy**2) / 2)

    F += -(1.0 / tau) * (F - Feq)

    # Apply boundary
    F[objects, :] = bndryF
    ux[objects] = 0
    uy[objects] = 0

    velocity = np.sqrt(ux**2+uy**2)
    velocity_img.set_array(velocity)
    velocity_img.set_clim(0, np.max(velocity))

    vorticity = (np.roll(ux, -1, axis=0) - np.roll(ux, 1, axis=0)) - (np.roll(uy, -1, axis=1) - np.roll(uy, 1, axis=1))
    vorticity[objects] = np.nan
    vorticity = np.ma.array(vorticity, mask=objects)

    vorticity_img.set_array(vorticity)
    vorticity_img.set_clim(-0.1, 0.1)

    return velocity_img, vorticity_img, cylinder_img

In [ ]:
anim = FuncAnimation(fig, update, frames=range(Nt), interval=1, blit=True, repeat=False)


In [ ]:
anim.save('animation.mp4', writer='ffmpeg', fps=60)